# Включение информации о характеристиках узлов с помощью многослойных перцептронов

In [1]:
# !pip install -q torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

In [2]:
from torch_geometric.datasets import Planetoid

# импортируем набор из PyTorch Geometric
dataset = Planetoid(root=".", name="Cora")

data = dataset[0]

# печатаем информацию о наборе данных
print(f'Набор данных: {dataset}')
print('---------------')
print(f'Количество графов: {len(dataset)}')
print(f'Количество узлов: {data.x.shape[0]}')
print(f'Количество признаков: {dataset.num_features}')
print(f'Количество классов: {dataset.num_classes}')

# печатаем информацию о графе
print(f'\nГраф:')
print('------')
print(f'Ребра являются ориентированными: {data.is_directed()}')
print(f'У графа есть изолированные узлы: {data.has_isolated_nodes()}')
print(f'У графа есть петли: {data.has_self_loops()}')

Набор данных: Cora()
---------------
Количество графов: 1
Количество узлов: 2708
Количество признаков: 1433
Количество классов: 7

Граф:
------
Ребра являются ориентированными: False
У графа есть изолированные узлы: False
У графа есть петли: False


Processing...
Done!


In [3]:
from torch_geometric.datasets import FacebookPagePage

# импортируем набор из PyTorch Geometric
dataset = FacebookPagePage(root=".")

data = dataset[0]

# печатаем информацию о наборе данных
print(f'Набор данных: {dataset}')
print('-----------------------')
print(f'Количество графов: {len(dataset)}')
print(f'Количество узлов: {data.x.shape[0]}')
print(f'Количество признаков: {dataset.num_features}')
print(f'Количество классов: {dataset.num_classes}')

# печатаем информацию о графе
print(f'\nГраф:')
print('------')
print(f'Ребра являются ориентированными: {data.is_directed()}')
print(f'У графа есть изолированные узлы: {data.has_isolated_nodes()}')
print(f'У графа есть петли: {data.has_self_loops()}')

# создаем маски
data.train_mask = range(18000)
data.val_mask = range(18001, 20000)
data.test_mask = range(20001, 22470)

Набор данных: FacebookPagePage()
-----------------------
Количество графов: 1
Количество узлов: 22470
Количество признаков: 128
Количество классов: 4

Граф:
------
Ребра являются ориентированными: False
У графа есть изолированные узлы: False
У графа есть петли: True


Processing...
Done!


## Набор данных Cora

In [4]:
import pandas as pd

dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

df_x = pd.DataFrame(data.x.numpy())
df_x['label'] = pd.DataFrame(data.y)
df_x

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2704,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [5]:
import torch
torch.manual_seed(0)
from torch.nn import Linear
import torch.nn.functional as F


def accuracy(y_pred, y_true):
    """Вычисляет правильность."""
    return torch.sum(y_pred == y_true) / len(y_true)


class MLP(torch.nn.Module):
    """Многослойный перцептрон"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.linear1 = Linear(dim_in, dim_h)
        self.linear2 = Linear(dim_h, dim_out)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        return F.log_softmax(x, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
                                     lr=0.01,
                                     weight_decay=5e-4)

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                                  data.y[data.val_mask])
                print(f'Эпоха {epoch:>3}:\n| Функция потерь на обуч. выборке: '
                      f'{loss:.3f} | Правильность на обуч. выборке: '
                      f'{acc*100:>5.2f}% \n| Функция потерь на валид. '
                      f'выборке: {val_loss:.2f} | Правильность на валид. '
                      f'выборке: {val_acc*100:.2f}%')

    @torch.no_grad()      
    def test(self, data):
        self.eval()
        out = self(data.x)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

# создаем модель MLP
mlp = MLP(dataset.num_features, 16, dataset.num_classes)
print(mlp)

# обучаем
mlp.fit(data, epochs=100)

# тестируем
cora_mlp_acc = mlp.test(data)
print(f'\nПравильность MLP на тесте (Cora): {cora_mlp_acc*100:.2f}%')

MLP(
  (linear1): Linear(in_features=1433, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=7, bias=True)
)
Эпоха   0:
| Функция потерь на обуч. выборке: 1.959 | Правильность на обуч. выборке: 14.29% 
| Функция потерь на валид. выборке: 2.00 | Правильность на валид. выборке: 12.40%
Эпоха  20:
| Функция потерь на обуч. выборке: 0.110 | Правильность на обуч. выборке: 100.00% 
| Функция потерь на валид. выборке: 1.46 | Правильность на валид. выборке: 49.40%
Эпоха  40:
| Функция потерь на обуч. выборке: 0.014 | Правильность на обуч. выборке: 100.00% 
| Функция потерь на валид. выборке: 1.44 | Правильность на валид. выборке: 51.00%
Эпоха  60:
| Функция потерь на обуч. выборке: 0.008 | Правильность на обуч. выборке: 100.00% 
| Функция потерь на валид. выборке: 1.40 | Правильность на валид. выборке: 53.80%
Эпоха  80:
| Функция потерь на обуч. выборке: 0.008 | Правильность на обуч. выборке: 100.00% 
| Функция потерь на валид. выборке: 1.37 | Правильность на валид. в

In [6]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        x = torch.sparse.mm(adjacency, x)
        return x

In [7]:
from torch_geometric.utils import to_dense_adj

adjacency = to_dense_adj(data.edge_index)[0]
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])

In [8]:
class VanillaGNN(torch.nn.Module):
    """Простая графовая нейронная сеть"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        return F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
                                     lr=0.01,
                                     weight_decay=5e-4)

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, adjacency)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                                  data.y[data.val_mask])
                print(f'Эпоха {epoch:>3}:\n| Функция потерь на обуч. выборке: '
                      f'{loss:.3f} | Правильность на обуч. выборке: '
                      f'{acc*100:>5.2f}% \n| Функция потерь на валид. '
                      f'выборке: {val_loss:.2f} | Правильность на валид. '
                      f'выборке: {val_acc*100:.2f}%')

    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data.x, adjacency)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

# создаем модель простой GNN
gnn = VanillaGNN(dataset.num_features, 16, dataset.num_classes)
print(gnn)

# обучаем 
gnn.fit(data, epochs=100)

# тестируем
cora_gnn_acc = gnn.test(data)
print(f'\nПравильность GNN на тесте (Cora): {cora_gnn_acc*100:.2f}%')

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=False)
  )
)
Эпоха   0:
| Функция потерь на обуч. выборке: 1.991 | Правильность на обуч. выборке: 15.71% 
| Функция потерь на валид. выборке: 2.11 | Правильность на валид. выборке: 9.40%
Эпоха  20:
| Функция потерь на обуч. выборке: 0.065 | Правильность на обуч. выборке: 99.29% 
| Функция потерь на валид. выборке: 1.47 | Правильность на валид. выборке: 76.80%
Эпоха  40:
| Функция потерь на обуч. выборке: 0.014 | Правильность на обуч. выборке: 100.00% 
| Функция потерь на валид. выборке: 2.11 | Правильность на валид. выборке: 75.40%
Эпоха  60:
| Функция потерь на обуч. выборке: 0.007 | Правильность на обуч. выборке: 100.00% 
| Функция потерь на валид. выборке: 2.22 | Правильность на валид. выборке: 75.40%
Эпоха  80:
| Функция потерь на обуч. выборке: 0.004 | Правильность на обуч. выборке: 100.0

## Набор данных Facebook Page-Page

In [9]:
# набор данных
dataset = FacebookPagePage(root=".")
data = dataset[0]
data.train_mask = range(18000)
data.val_mask = range(18001, 20000)
data.test_mask = range(20001, 22470)

# матрица смежности
adjacency = to_dense_adj(data.edge_index)[0]
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [10]:
# MLP
mlp = MLP(dataset.num_features, 16, dataset.num_classes)
print(mlp)
mlp.fit(data, epochs=100)
fb_mlp_acc = mlp.test(data)
print(f'\nПравильность MLP на тесте (Facebook): {fb_mlp_acc*100:.2f}%\n')

# GCN
gnn = VanillaGNN(dataset.num_features, 16, dataset.num_classes)
print(gnn)
gnn.fit(data, epochs=100)
fb_gnn_acc = gnn.test(data)
print(f'\nПравильность GNN на тесте (Facebook): {fb_gnn_acc*100:.2f}%')

MLP(
  (linear1): Linear(in_features=128, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=4, bias=True)
)
Эпоха   0:
| Функция потерь на обуч. выборке: 1.401 | Правильность на обуч. выборке: 28.11% 
| Функция потерь на валид. выборке: 1.40 | Правильность на валид. выборке: 28.91%
Эпоха  20:
| Функция потерь на обуч. выборке: 0.671 | Правильность на обуч. выборке: 73.47% 
| Функция потерь на валид. выборке: 0.68 | Правильность на валид. выборке: 72.94%
Эпоха  40:
| Функция потерь на обуч. выборке: 0.579 | Правильность на обуч. выборке: 76.95% 
| Функция потерь на валид. выборке: 0.61 | Правильность на валид. выборке: 74.89%
Эпоха  60:
| Функция потерь на обуч. выборке: 0.549 | Правильность на обуч. выборке: 78.20% 
| Функция потерь на валид. выборке: 0.60 | Правильность на валид. выборке: 75.59%
Эпоха  80:
| Функция потерь на обуч. выборке: 0.533 | Правильность на обуч. выборке: 78.76% 
| Функция потерь на валид. выборке: 0.60 | Правильность на валид. выборк

In [11]:
data_dict = {'MLP': [f'{cora_mlp_acc*100:.2f}%', 
                     f'{fb_mlp_acc*100:.2f}%'], 
             'GNN': [f'{cora_gnn_acc*100:.2f}%', 
                     f'{fb_gnn_acc*100:.2f}%']}
metrics_df = pd.DataFrame(data_dict, 
                          index=['Cora', 'Facebook'])
metrics_df

,MLP,GNN
Cora,53.40%,76.60%
Facebook,75.33%,85.10%
